#### The first thing we'll do is installing required packages. Press Shift + Enter to make the cell run

In [1]:
pip install panda

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install yahoo-fantasy-api

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install nested-lookup

Note: you may need to restart the kernel to use updated packages.


####  After pressing Shift+Enter the below cell. Click and open the authorization URL.
#### Accept and copy the code to the "Enter verifier" field. The error message will show "KeyError: 'access_token" if the code is not correct, Shift-Enter this cell again.

In [4]:
import yahoo_fantasy_api as yfa
import pandas as pd
from yahoo_oauth import OAuth2
import json

creds = {'consumer_key': 'dj0yJmk9bDAwMjU4QnlycUs1JmQ9WVdrOVNEWjZOVUU0ZHpnbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PWU3',\
         'consumer_secret': 'a317d5b0cddde7f6c572de738f095d1732f11abb'}
with open('oauth2.json', "w") as f:
    f.write(json.dumps(creds))
oauth = OAuth2(None, None, from_file='oauth2.json')

[2021-03-31 07:37:53,800 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-31 07:37:53,802 DEBUG] [yahoo_oauth.oauth.handler] AUTHORIZATION URL : https://api.login.yahoo.com/oauth2/request_auth?redirect_uri=oob&response_type=code&client_id=dj0yJmk9bDAwMjU4QnlycUs1JmQ9WVdrOVNEWjZOVUU0ZHpnbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PWU3


Enter verifier : pw83nms


#### Find the league_id

In [5]:
gm = yfa.game.Game(oauth, 'nba')
gm.league_ids(year=2020)

['402.l.20641', '402.l.28035']

#### Replace your league_id to parameter field. An example is '402.l.28035'

In [6]:
lg = gm.to_league('402.l.28035')

#### Before running it, change the week you want to ccess and obtain league stats.

In [7]:
from nested_lookup import nested_lookup
matchup = lg.matchups(week=13)

stat_name = ["Team","FG%","FT%","3PTM","PTS","REB","AST","ST","BLK","TO","Score"]
league_stat = pd.DataFrame(columns=stat_name)


#Obtain the league stats

name = nested_lookup('name', matchup)
team_stats = nested_lookup('team_stats', matchup)
team_points = nested_lookup('team_points', matchup)

no_team = len(lg.teams().keys())
for i in range (no_team):
    stat = nested_lookup('value', team_stats[i])
    point = nested_lookup('total', team_points[i])
    stat = list(map(float,[ x for x in stat if "/" not in x ]))
    #stat = list(map(int))
    point = list(map(float,point))
    full_stat = [name[i+1]] + stat + point
    league_stat.loc[len(league_stat)] = full_stat
    
#Calculate the roto ranks
league_stat['FG%_rank']=league_stat['FG%'].rank(ascending=True, method='min')
league_stat['FT%_rank']=league_stat['FT%'].rank(ascending=True, method='min')    
league_stat['3PTM_rank']=league_stat['3PTM'].rank(ascending=True, method='min')    
league_stat['PTS_rank']=league_stat['PTS'].rank(ascending=True, method='min')    
league_stat['REB_rank']=league_stat['REB'].rank(ascending=True, method='min')    
league_stat['AST_rank']=league_stat['AST'].rank(ascending=True, method='min')    
league_stat['ST_rank']=league_stat['ST'].rank(ascending=True, method='min')    
league_stat['BLK_rank']=league_stat["BLK"].rank(ascending=True, method='min')      
league_stat['TO_rank']=league_stat['TO'].rank(ascending=False, method='min')    
league_stat['Total']= league_stat['FG%_rank']+league_stat['FT%_rank']+league_stat['3PTM_rank']+league_stat['REB_rank']+league_stat['PTS_rank'] + \
 league_stat['AST_rank']  + league_stat['ST_rank'] + league_stat['BLK_rank'] + league_stat['TO_rank']
league_stat['Roto_rank']=league_stat['Total'].rank(ascending=True, method='min') 

league_stat


[2021-03-31 07:38:07,382 INFO] [numexpr.utils.utils._init_num_threads] Note: NumExpr detected 56 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2021-03-31 07:38:07,384 INFO] [numexpr.utils.utils._init_num_threads] NumExpr defaulting to 8 threads.


,Team,FG%,FT%,3PTM,PTS,REB,AST,ST,BLK,TO,...,FT%_rank,3PTM_rank,PTS_rank,REB_rank,AST_rank,ST_rank,BLK_rank,TO_rank,Total,Roto_rank
0,Etsuwa,0.454,0.738,38.0,329.0,114.0,73.0,21.0,9.0,36.0,...,5.0,5.0,2.0,2.0,5.0,8.0,1.0,16.0,54.0,1.0
1,Lêbông James,0.509,0.848,45.0,312.0,116.0,62.0,18.0,12.0,31.0,...,16.0,10.0,1.0,3.0,2.0,3.0,3.0,19.0,75.0,4.0
2,Lyon King,0.441,0.691,43.0,350.0,134.0,62.0,29.0,20.0,47.0,...,1.0,9.0,3.0,6.0,2.0,15.0,12.0,9.0,63.0,2.0
3,huy's Team,0.433,0.864,46.0,393.0,123.0,82.0,20.0,13.0,34.0,...,18.0,11.0,9.0,4.0,7.0,5.0,4.0,17.0,78.0,6.0
4,Dendi fangay,0.460,0.729,57.0,426.0,167.0,126.0,23.0,14.0,56.0,...,3.0,16.0,11.0,10.0,20.0,10.0,6.0,3.0,91.0,9.0
5,Strongest Avenger,0.433,0.787,29.0,407.0,169.0,114.0,32.0,20.0,59.0,...,10.0,2.0,10.0,12.0,18.0,18.0,12.0,2.0,87.0,8.0
6,Hades' Angel,0.431,0.813,62.0,352.0,146.0,58.0,27.0,33.0,27.0,...,14.0,20.0,4.0,7.0,1.0,13.0,19.0,20.0,100.0,14.0
7,Kuatlam Kaveland,0.451,0.730,39.0,389.0,191.0,90.0,21.0,34.0,45.0,...,4.0,7.0,7.0,18.0,9.0,8.0,20.0,11.0,92.0,10.0
8,Hammer of Sigmar,0.453,0.857,57.0,463.0,175.0,86.0,20.0,13.0,45.0,...,17.0,16.0,17.0,14.0,8.0,5.0,4.0,11.0,101.0,15.0
9,Đà Nẵng Heat,0.544,0.714,32.0,497.0,220.0,98.0,31.0,24.0,53.0,...,2.0,4.0,18.0,19.0,12.0,17.0,15.0,6.0,112.0,17.0


#### Make the weekly stats table

In [12]:
def highlight_max(s):
    is_max=(s == s.nlargest(3)[0]) | (s == s.nlargest(3)[1]) | (s == s.nlargest(3)[2]) 
    return ['background-color: lightgreen' if v else '' for v in is_max]
   
def highlight_min(s):
    is_min =(s == s.nsmallest(3)[0]) | (s == s.nsmallest(3)[1]) | (s == s.nsmallest(3)[2])
    return ['background-color: pink' if v else '' for v in is_min]
def highlight_max_to(s):
    is_max=(s == s.nlargest(3)[0]) | (s == s.nlargest(3)[1]) | (s == s.nlargest(3)[2]) 
    return ['background-color: pink' if v else '' for v in is_max]
def highlight_min_to(s):
    is_min =(s == s.nsmallest(3)[0]) | (s == s.nsmallest(3)[1]) | (s == s.nsmallest(3)[2])
    return ['background-color: lightgreen' if v else '' for v in is_min]

head_to_head_stats = league_stat.iloc[:, 0:11]
head_to_head_stats.set_index('Team', inplace=True, drop=True)
head_to_head_stats.index.names = ['']

head_to_head_stats.style.set_precision(0).format({
    'FG%': '{:,.3f}'.format,
    'FT%': '{:,.3f}'.format,
}).apply(highlight_max, subset=['FG%','FT%','3PTM','PTS','REB','AST','ST','BLK']) \
.apply(highlight_min,subset=['FG%','FT%','3PTM','PTS','REB','AST','ST','BLK'])\
.apply(highlight_max_to, subset=['TO']).apply(highlight_min_to, subset=['TO'])\
.set_caption(name[0]+" Weekly Stats ").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '15px')
    ]
}])



,FG%,FT%,3PTM,PTS,REB,AST,ST,BLK,TO,Score
,,,,,,,,,,
Etsuwa,0.454,0.738,38,329,114,73,21,9,36,3
Lêbông James,0.509,0.848,45,312,116,62,18,12,31,6
Lyon King,0.441,0.691,43,350,134,62,29,20,47,4
huy's Team,0.433,0.864,46,393,123,82,20,13,34,5
Dendi fangay,0.460,0.729,57,426,167,126,23,14,56,5
Strongest Avenger,0.433,0.787,29,407,169,114,32,20,59,4
Hades' Angel,0.431,0.813,62,352,146,58,27,33,27,4
Kuatlam Kaveland,0.451,0.730,39,389,191,90,21,34,45,5
Hammer of Sigmar,0.453,0.857,57,463,175,86,20,13,45,3


#### Make the roto rank table

In [11]:
def highlight_max(s):
    is_max=(s == s.nlargest(3)[0]) | (s == s.nlargest(3)[1]) | (s == s.nlargest(3)[2]) 
    return ['background-color: lightgreen' if v else '' for v in is_max]
def highlight_min(s):
    is_min =(s == s.nsmallest(3)[0]) | (s == s.nsmallest(3)[1]) | (s == s.nsmallest(3)[2])
    return ['background-color: pink' if v else '' for v in is_min]

stat_name = ["Team","FG%","FT%","3PTM","PTS","REB","AST","ST","BLK","TO","Total","Rank"]



roto_stats = league_stat.iloc[:,0:1].join(league_stat.iloc[:,11:22])
roto_stats.columns = stat_name
roto_stats.set_index('Team', inplace=True, drop=True)
roto_stats.index.names = ['']
roto_stats.sort_values(by=['Total'], inplace=True, ascending=False)

roto_stats.style.set_precision(0).apply(highlight_max) \
.apply(highlight_min).set_caption(name[0]+" Roto Rank ").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '15px')
    ]
}])


,FG%,FT%,3PTM,PTS,REB,AST,ST,BLK,TO,Total,Rank
,,,,,,,,,,,
Kim Long Bang,16,19,19,16,13,16,18,10,13,140,20
KarlAnthonyTowns,17,15,13,15,17,15,20,17,8,137,19
Cá Voi Lưng Gù,20,10,3,20,19,11,13,17,3,116,18
Đà Nẵng Heat,19,2,4,18,19,12,17,15,6,112,17
Zenith Blade,15,9,15,13,5,19,5,11,13,105,16
Hammer of Sigmar,9,17,16,17,14,8,5,4,11,101,15
Hades' Angel,2,14,20,4,7,1,13,19,20,100,14
Red Devil,5,13,14,12,11,13,16,7,6,97,12
RipCity Wanderers,11,7,16,19,15,6,11,7,5,97,12
